In [5]:
import glob
import re
import pandas as pd
from IPython.core.display import display

In [107]:
coref_df = pd.DataFrame(columns=['Doc_ID', 'Text_ID', 'Text', 'Coref_Clusters'])
print(coref_df)


Empty DataFrame
Columns: [Doc_ID, Text_ID, Text, Coref_Clusters]
Index: []


In [99]:
def get_coref_clusters(path, document_id):
    temp_df = pd.DataFrame(columns=['Doc_ID', 'Text_ID', 'Text', 'Coref_Clusters'])

    with open(path, 'r') as file:
        lines = file.readlines()

        text_id = 0
        text = []
        coref_clusters = {}
        temp_clusters = {}

        for line in lines:
            if line == '\n':
                temp_df = temp_df.append({'Doc_ID': document_id,
                                          'Text_ID': document_id + '-' + text_id,
                                          'Text': text,
                                          'Coref_Clusters': list(coref_clusters.values())},
                                         ignore_index=True)
                text = []
                coref_clusters = {}
                temp_clusters = {}
            elif not line.startswith('#'):
                columns = line.split()
                text_id = columns[1]
                text.append(columns[3])
                coref_column = columns[12]
                index = int(columns[2])
                if not coref_column == '-':
                    corefs = coref_column.split('|')
                    for coref in corefs:
                        key = coref.strip('()')
                        if re.match(r'\([0-9]+\)', coref):
                            coref_clusters.setdefault(key, []).append([index])
                        if re.match(r'^\([0-9]+$', coref):
                            temp_clusters.setdefault(key, []).append(index)
                        if re.match(r'^[0-9]+\)$', coref):
                            temp_clusters.setdefault(key, []).append(index)
                            cluster_span = temp_clusters.pop(key)
                            coref_clusters.setdefault(key, []).append(cluster_span)
    return temp_df

In [108]:
for path in glob.iglob('data/datasets/suc-core-mod/' + '*.conll'):
    filename = path.split('/')[-1]
    document_id = filename.split('-')[0]
    df = get_coref_clusters(path, document_id)
    coref_df = coref_df.append(df)

display(coref_df)
coref_df.to_pickle('data/coref.pkl')

,Doc_ID,Text_ID,Text,Coref_Clusters
0,kk14,kk14-001,"[Det, blev, en, vidrig, resa, men, eftersom, d...",[[[13]]]
1,kk14,kk14-002,"[Vi, reste, bland, indier, och, de, hindrade, ...","[[[1], [8], [12]], [[4], [6]]]"
2,kk14,kk14-003,"[När, vi, bad, att, få, låna, en, gryta, blev,...","[[[2]], [[10]]]"
3,kk14,kk14-004,"[Vi, var, inte, rena, nog, att, röra, vid, der...","[[[1]], [[9]]]"
4,kk14,kk14-005,"[Två, stormar, överföll, oss, ,, några, dog, o...",[[[4]]]
...,...,...,...,...
245,ea12,ea12-264,"[I, regel, väljer, den, dock, brandskadade, tr...",[[[4]]]
246,ea12,ea12-265,"[Larven, ,, som, kan, bli, 40, mm, lång, ,, gn...",[[[1]]]
247,ea12,ea12-266,"[Den, förpuppas, också, där, och, tar, sig, ut...","[[[1], [7]]]"
248,ea12,ea12-267,"[Hela, utvecklingen, tar, 2-3, år, .]",[]


In [ ]:
# TODO: get text and translate. check tokenization? create alignments.
#       create aligned clusters.